In [1]:
from pyspark.sql import SparkSession

In [2]:
# Criando uma sessão spark
spark = SparkSession.builder.appName("prob_SISU").getOrCreate()

23/11/28 08:35:25 WARN Utils: Your hostname, daniel-VJFE43F11X-XXXXXX resolves to a loopback address: 127.0.1.1; using 192.168.0.157 instead (on interface wlo1)
23/11/28 08:35:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/28 08:35:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Lendo o arquivo csv
dados = spark.read.csv("../data/raw/chamada_regular_sisu_2022_1.csv", header=True, sep="|", encoding="latin1")

In [4]:
# Observando o estado inicial dos dados
dados.show(5)

23/11/28 08:35:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------+-----+---------------+----------+--------------------+---------+------+-------------+--------------------+---------+----------------+------------+--------------------+-----------+-------+----------------+-------+---------------------+------------------+---------------------+----------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+-------------+--------------+--------------------+----+---------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+--------------+
| ANO|EDICAO|ETAPA|       DS_ETAPA|CODIGO_IES|            NOME_IES|SIGLA_IES|UF_IES|CODIGO_CAMPUS|         NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|CODIGO_CURSO|          NOME_CURSO|       GRAU|  TURNO|DS_PERIODICIDADE|TP_COTA|TIPO_MOD_CONCORRENCIA|  MOD_CONCORRENCIA|QT_VAGAS_CONC

In [5]:
# Permitindo usar SQL nos dados
dados.createOrReplaceTempView("dados")

A partir dos dados iniciais do SISU, vamos fazer uma análise exploratória dos dados para responder as seguintes perguntas: 
- Qual a porcentagem de aprovados?
- Qual a maior e menor nota de aprovação?
- Dos aprovados, quantos porcento foram aceitos na primeira opção? E na segunda?
- Qual a probabilidade de ser aprovado na segunda opção dado que foi reprovado na primeira?
- Qual a porcentagem de cotistas aplicantes?
- Destes, qual a porcentagem de aprovados?
- Qual a porcentagem de cotistas aprovados na primeira opção? E na segunda?
- Qual a probabilidade de ser aprovado na primeira opção sendo um cotista?
- Há diferença na probabilidade de ser aprovado em um curso dado as modalidades de concorrência (cotas, ampla concorrência, etc)?
- Há alguma diferença na probabilidade de ser aprovado em um curso dado o seu turno (Vespertino, Matutino, Noturno, Integral, EAD)?
- Há alguma diferença na probabilidade de ser aprovado em um curso dado o seu grau? (Área Básica de ingresso, Bacharelado, Licenciatura, Tecnológico)

# Qual a taxa de aprovação do ENEM?
Obs: A taxa é considerando primeira e segunda escolha.

In [6]:
# Calculando a taxa de aprovados
qtd_aprovados = spark.sql("SELECT ROUND(COUNT(*), 2) \
                           FROM dados \
                           WHERE APROVADO = 'S'").collect()[0][0]

# Calculando o total de registros
total_de_registros = spark.sql("SELECT COUNT(*) FROM dados").collect()[0][0]

print(f"Taxa de aprovados: {round(qtd_aprovados / total_de_registros * 100, 2)}%")

Taxa de aprovados: 10.18%


# Qual a maior e a menor nota de aprovação?

In [7]:
# Buscando a menor nota de candidatos aprovados
menor_nota = spark.sql("SELECT MIN(NOTA_CANDIDATO) \
                        FROM dados \
                        WHERE APROVADO = 'S'").collect()[0][0]

print(f"Menor nota de candidatos aprovados: {menor_nota}")

Menor nota de candidatos aprovados: 208,79


In [8]:
# Buscando a maior nota de candidatos aprovados
maior_nota = spark.sql("SELECT MAX(NOTA_CANDIDATO) \
                        FROM dados \
                        WHERE APROVADO = 'S'").collect()[0][0]

print(f"Maior nota de candidatos aprovados: {maior_nota}")

Maior nota de candidatos aprovados: 953,14


In [9]:
# Verificando o terceiro quartil das notas de candidatos aprovados
terceiro_quartil = spark.sql("SELECT PERCENTILE(NOTA_CANDIDATO, 0.75) \
                              FROM dados \
                              WHERE APROVADO = 'S'").collect()[0][0]

print(f"75% das notas dos aprovados está abaixo de {terceiro_quartil}")

75% das notas dos aprovados está abaixo de 686.0


In [10]:
# Verificando o primeiro quartil das notas dos candidatos aprovados
primeiro_quartil = spark.sql("SELECT PERCENTILE(NOTA_CANDIDATO, 0.25) \
                              FROM dados \
                              WHERE APROVADO = 'S'").collect()[0][0]

print(f"25% das notas dos aprovados está abaixo de {primeiro_quartil}")


25% das notas dos aprovados está abaixo de 577.0


# Dos aprovados, quantos porcento foram aceitos na primeira opção? E na segunda?

In [11]:
# Buscando o número de aprovados na primeira opção
aprovados_primeira_opcao = spark.sql("SELECT COUNT(*) \
                                      FROM dados \
                                      WHERE APROVADO = 'S' \
                                      AND OPCAO = 1").collect()[0][0]

print(f"Do total de aprovados, {round(aprovados_primeira_opcao / qtd_aprovados * 100, 2)}% foram aceitos na primeira opção")

Do total de aprovados, 63.85% foram aceitos na primeira opção


In [12]:
# Calculando os aprovados somente na segunda opção
aprovados_segunda_opcao = spark.sql("SELECT COUNT(*) \
                                     FROM dados \
                                     WHERE APROVADO = 'S' \
                                     AND OPCAO = 2").collect()[0][0]

print(f"Do total de aprovados, {round(aprovados_segunda_opcao / qtd_aprovados * 100, 2)}% foram aceitos na segunda opção")

Do total de aprovados, 36.15% foram aceitos na segunda opção


# Qual a probabilidade de ser aprovado na segunda opção dado que foi reprovado na primeira?

In [13]:
# Selecionando os reprovados na primeira opção
reprovados_primeira_opcao = spark.sql("SELECT CPF, INSCRITO \
                                       FROM dados \
                                       WHERE APROVADO = 'N' \
                                       AND OPCAO = 1")

# Selecionando os aprovados
aprovados = spark.sql("SELECT CPF, INSCRITO \
                                     FROM dados \
                                     WHERE APROVADO = 'S'")

# Tornando possivel usar SQL nos dados
reprovados_primeira_opcao.createOrReplaceTempView("reprovados_primeira_opcao")
aprovados.createOrReplaceTempView("aprovados")

# Buscando os reprovados na primeira opção que foram aprovados na segunda opção
qtd_reprovados_1_aprovados = spark.sql("SELECT COUNT(*) \
                                          FROM reprovados_primeira_opcao \
                                          INNER JOIN aprovados \
                                          ON reprovados_primeira_opcao.CPF = aprovados.CPF \
                                          AND reprovados_primeira_opcao.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Contando os aprovados na primeira opção
qtd_reprovados_primeira_opcao = reprovados_primeira_opcao.count()

# Contando os aprovados na segunda opção
qtd_aprovados = aprovados.count()

In [14]:
# Calculando a probabilidade de ser reprovado na primeira opcao
probabilidade_reprovado_1_opcao = qtd_reprovados_primeira_opcao / total_de_registros

# Calculando a probabilidade de ser reprovado na primeira opcao e aprovado na segunda
probabilidade_reprovado_1_aprovado_2 = qtd_reprovados_1_aprovados / total_de_registros

# Calculando a probabilidade de ser aprovado na segunda opcao dado que foi reprovado na primeira
probabilidade_aprovado_2_reprovado_1 = probabilidade_reprovado_1_aprovado_2 / probabilidade_reprovado_1_opcao

print(f"A probabilidade de ser aprovado na segunda opção dado que foi reprovado na primeira é de {round(probabilidade_aprovado_2_reprovado_1 * 100, 2)}%")

A probabilidade de ser aprovado na segunda opção dado que foi reprovado na primeira é de 8.11%


# Qual a porcentagem de cotistas aplicantes?

In [15]:
# Buscando a quantidade de candidatos que usaram algum tipo de cota
qtd_cotistas = spark.sql("SELECT COUNT(*) \
                          FROM dados \
                          WHERE TP_COTA IS NOT NULL").collect()[0][0]


In [16]:
# Calculando a porcentagem de candidatos que usaram algum tipo de cota
porcentagem_cotistas = round(qtd_cotistas / total_de_registros * 100, 2)

print(f"{porcentagem_cotistas}% dos candidatos usaram algum tipo de cota")

43.1% dos candidatos usaram algum tipo de cota


# Qual a porcentagem de cotistas aprovados?

In [17]:
# Buscando a quantidade de candidatos que usaram algum tipo de cota e foram aprovados
qtd_cotistas_aprovados = spark.sql("SELECT COUNT(*) \
                                    FROM dados \
                                    WHERE TP_COTA IS NOT NULL AND APROVADO = 'S'").collect()[0][0]

In [18]:
# Calculando a porcentagem de candidatos que usaram algum tipo de cota e foram aprovados
porcentagem_cotistas_aprovados = round(qtd_cotistas_aprovados / qtd_cotistas * 100, 2)

print(f"{porcentagem_cotistas_aprovados}% dos candidatos que usaram algum tipo de cota foram aprovados")

9.69% dos candidatos que usaram algum tipo de cota foram aprovados


# Qual a porcentagem de cotistas aprovados na primeira opção? E na segunda?

In [20]:
# Buscando a quantidade de candidatos que usaram algum tipo de cota e foram aprovados na primeira opção
qtd_cotistas_aprovados_primeira_opcao = spark.sql("SELECT COUNT(*) \
                                                    FROM dados \
                                                    WHERE TP_COTA IS NOT NULL \
                                                    AND APROVADO = 'S' \
                                                    AND OPCAO = 1").collect()[0][0]

# Calculando a porcentagem de candidatos que usaram algum tipo de cota e foram aprovados na primeira opção
porcentagem_cotistas_aprovados_primeira_opcao = round(qtd_cotistas_aprovados_primeira_opcao / qtd_cotistas_aprovados * 100, 2)

print(f"{porcentagem_cotistas_aprovados_primeira_opcao}% dos candidatos cotistas foram aprovados na primeira opção")

64.08% dos candidatos cotistas foram aprovados na primeira opção


In [21]:

# Buscando a quantidade de candidatos que usaram algum tipo de cota e foram aprovados na segunda opção
qtd_cotistas_aprovados_segunda_opcao = spark.sql("SELECT COUNT(*) \
                                                   FROM dados \
                                                   WHERE TP_COTA IS NOT NULL \
                                                   AND APROVADO = 'S' \
                                                   AND OPCAO = 2").collect()[0][0]

# Calculando a porcentagem de candidatos que usaram algum tipo de cota e foram aprovados na segunda opção
porcentagem_cotistas_aprovados_segunda_opcao = round(qtd_cotistas_aprovados_segunda_opcao / qtd_cotistas_aprovados * 100, 2)

print(f"{porcentagem_cotistas_aprovados_segunda_opcao}% dos candidatos cotistas foram aprovados na segunda opção")

35.92% dos candidatos cotistas foram aprovados na segunda opção


# Qual a probabilidade de ser aprovado na primeira opção sendo um cotista?

In [26]:
# Buscando os dados dos cotistas
cotistas = spark.sql("SELECT CPF, INSCRITO \
                      FROM dados \
                      WHERE TP_COTA IS NOT NULL")

# Buscando os aprovados na primeira opção
aprovados_primeira_opcao = spark.sql("SELECT CPF, INSCRITO \
                                      FROM dados \
                                      WHERE APROVADO = 'S' \
                                      AND OPCAO = 1")

# Tornando possivel usar SQL nos dados
cotistas.createOrReplaceTempView("cotistas")
aprovados_primeira_opcao.createOrReplaceTempView("aprovados_primeira_opcao")

# Buscando os cotistas que foram aprovados na primeira opção
qtd_cotistas_aprovados_primeira_opcao = spark.sql("SELECT COUNT(*) \
                                                    FROM cotistas \
                                                    INNER JOIN aprovados_primeira_opcao \
                                                    ON cotistas.CPF = aprovados_primeira_opcao.CPF \
                                                    AND cotistas.INSCRITO = aprovados_primeira_opcao.INSCRITO").collect()[0][0]

# Calculando a quantidade de cotistas
qtd_cotistas = cotistas.count()

# Calculando a quantidade de aprovações na primeira opção
qtd_aprovados_primeira_opcao = aprovados_primeira_opcao.count()

In [35]:
# Calculando a probabilidade de ser cotistas
probabilidade_cotista = qtd_cotistas / total_de_registros

# Calculando a probabilidade de ser aprovado na primeira opção e ser cotista
probabilidade_aprovado_primeira_opcao_cotista = qtd_cotistas_aprovados_primeira_opcao / total_de_registros

# Calculando a probabilidade de ser aprova na primeira opção dado que é cotista
probabilidade_aprovado_primeira_opcao_dado_cotista = probabilidade_aprovado_primeira_opcao_cotista / probabilidade_cotista

print(f"A probabilidade de ser aprovado na primeira opção dado que é cotista é de {round(probabilidade_aprovado_primeira_opcao_dado_cotista * 100, 2)}%")



A probabilidade de ser aprovado na primeira opção dado que é cotista é de 11.82%


# Há diferença na probabilidade de ser aprovado em um curso dado as modalidades de concorrência (cotas, ampla concorrência, etc)?

In [37]:
# Buscando os candidatos que nãi usaram nenhum tipo de cota
ampla_concorrencia = spark.sql("SELECT CPF, INSCRITO \
                                FROM dados \
                                WHERE TP_COTA IS NULL")

# Tornando possivel usar SQL nos dados
ampla_concorrencia.createOrReplaceTempView("ampla_concorrencia")

# Buscando os candidatos que não usaram nenhum tipo de cota e foram aprovados na primeira opção
qtd_ampla_concorrencia_aprovados_primeira_opcao = spark.sql("SELECT COUNT(*) \
                                                             FROM ampla_concorrencia \
                                                             INNER JOIN aprovados_primeira_opcao \
                                                             ON ampla_concorrencia.CPF = aprovados_primeira_opcao.CPF \
                                                             AND ampla_concorrencia.INSCRITO = aprovados_primeira_opcao.INSCRITO").collect()[0][0]

# Calculando a quantidade de candidatos que não usaram nenhum tipo de cota
qtd_ampla_concorrencia = ampla_concorrencia.count()

In [38]:
# Calculando a probabilidade de ser aprovado na primeira opção e não ser cotista
probabilidade_aprovado_primeira_opcao_ampla_concorrencia = qtd_ampla_concorrencia_aprovados_primeira_opcao / total_de_registros

# Calculando a probabilidade de estar na ampla concorrência
probabilidade_ampla_concorrencia = qtd_ampla_concorrencia / total_de_registros

# Calculando a probabilidade de ser aprovado na primeira opção dado que não é cotista
probabilidade_aprovado_primeira_opcao_dado_ampla_concorrencia = probabilidade_aprovado_primeira_opcao_ampla_concorrencia / probabilidade_ampla_concorrencia

print(f"A probabilidade de ser aprovado na primeira opção dado que não é cotista é de {round(probabilidade_aprovado_primeira_opcao_dado_ampla_concorrencia * 100, 2)}%")

A probabilidade de ser aprovado na primeira opção dado que não é cotista é de 13.0%


In [39]:
print(f"A probabilidade de ser aprovado na primeira opção dado que é cotista é de {round(probabilidade_aprovado_primeira_opcao_dado_cotista * 100, 2)}%")

A probabilidade de ser aprovado na primeira opção dado que é cotista é de 11.82%


# Há alguma diferença na probabilidade de ser aprovado em um curso dado o seu turno? (Vespertino, Matutino, Noturno, Integral, EAD)

In [47]:
# Buscando os candidatos que aplicaram para cursos matutinos
matutino = spark.sql("SELECT CPF, INSCRITO \
                      FROM dados \
                      WHERE TURNO = 'Matutino'")

# Buscando os candidatos que aplicaram para cursos vespertinos
vespertino = spark.sql("SELECT CPF, INSCRITO \
                        FROM dados \
                        WHERE TURNO = 'Vespertino'")

# Buscando os candidatos que aplicaram para cursos noturnos
noturno = spark.sql("SELECT CPF, INSCRITO \
                     FROM dados \
                     WHERE TURNO = 'Noturno'")

# Buscando os candidatos que aplicaram para cursos integrais
integral = spark.sql("SELECT CPF, INSCRITO \
                      FROM dados \
                      WHERE TURNO = 'Integral'")

# Buscando os candidatos que aplicaram para cursos EaD
ead = spark.sql("SELECT CPF, INSCRITO \
                 FROM dados \
                 WHERE TURNO = 'EaD'")

# Buscando os candidatos aprovados
aprovados = spark.sql("SELECT CPF, INSCRITO \
                       FROM dados \
                       WHERE APROVADO = 'S'")

# Tornando possivel usar SQL nos dados
matutino.createOrReplaceTempView("matutino")
vespertino.createOrReplaceTempView("vespertino")
noturno.createOrReplaceTempView("noturno")
integral.createOrReplaceTempView("integral")
ead.createOrReplaceTempView("ead")
aprovados.createOrReplaceTempView("aprovados")

# Buscando a quantidade de candidatos que aplicaram para cursos matutinos e foram aprovados
qtd_matutino_aprovado = spark.sql("SELECT COUNT(*) \
                                   FROM matutino \
                                   INNER JOIN aprovados \
                                   ON matutino.CPF = aprovados.CPF \
                                   AND matutino.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Buscando a quantidade de candidatos que aplicaram para cursos vespertinos e foram aprovados
qtd_vespertino_aprovado = spark.sql("SELECT COUNT(*) \
                                     FROM vespertino \
                                     INNER JOIN aprovados \
                                     ON vespertino.CPF = aprovados.CPF \
                                     AND vespertino.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Buscando a quantidade de candidatos que aplicaram para cursos noturnos e foram aprovados
qtd_noturno_aprovado = spark.sql("SELECT COUNT(*) \
                                  FROM noturno \
                                  INNER JOIN aprovados \
                                  ON noturno.CPF = aprovados.CPF \
                                  AND noturno.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Buscando a quantidade de candidatos que aplicaram para cursos integrais e foram aprovados
qtd_integral_aprovado = spark.sql("SELECT COUNT(*) \
                                   FROM integral \
                                   INNER JOIN aprovados \
                                   ON integral.CPF = aprovados.CPF \
                                   AND integral.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Buscanco a quantidade de candidatos que aplicaram para cursos EaD e foram aprovados
qtd_ead_aprovado = spark.sql("SELECT COUNT(*) \
                              FROM ead \
                              INNER JOIN aprovados \
                              ON ead.CPF = aprovados.CPF \
                              AND ead.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Calculando a quantidade de candidatos de cada turno
qtd_matutino = matutino.count()
qtd_vespertino = vespertino.count()
qtd_noturno = noturno.count()
qtd_integral = integral.count()
qtd_ead = ead.count()

In [48]:
# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso matutino
probabilidade_aprovado_matutino = qtd_matutino_aprovado / total_de_registros

# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso vespertino
probabilidade_aprovado_vespertino = qtd_vespertino_aprovado / total_de_registros

# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso noturno
probabilidade_aprovado_noturno = qtd_noturno_aprovado / total_de_registros

# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso integral
probabilidade_aprovado_integral = qtd_integral_aprovado / total_de_registros

# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso EaD
probabilidade_aprovado_ead = qtd_ead_aprovado / total_de_registros

# Calculando a probabilidade de escolher um curso matutino
probabilidade_matutino = qtd_matutino / total_de_registros

# Calculando a probabilidade de escolher um curso vespertino
probabilidade_vespertino = qtd_vespertino / total_de_registros

# Calculando a probabilidade de escolher um curso noturno
probabilidade_noturno = qtd_noturno / total_de_registros

# Calculando a probabilidade de escolher um curso integral
probabilidade_integral = qtd_integral / total_de_registros

# Calculando a probabilidade de escolher um curso EaD
probabilidade_ead = qtd_ead / total_de_registros

# Calculando a probabilidade de ser aprovado dado os diferentes turnos
probabilidade_aprovado_dado_matutino = probabilidade_aprovado_matutino / probabilidade_matutino
probabilidade_aprovado_dado_vespertino = probabilidade_aprovado_vespertino / probabilidade_vespertino
probabilidade_aprovado_dado_noturno = probabilidade_aprovado_noturno / probabilidade_noturno
probabilidade_aprovado_dado_integral = probabilidade_aprovado_integral / probabilidade_integral
probabilidade_aprovado_dado_ead = probabilidade_aprovado_ead / probabilidade_ead

print(f"A probabilidade de ser aprovado dado que escolheu um curso matutino é de {round(probabilidade_aprovado_dado_matutino * 100, 2)}%")
print(f"A probabilidade de ser aprovado dado que escolheu um curso vespertino é de {round(probabilidade_aprovado_dado_vespertino * 100, 2)}%")
print(f"A probabilidade de ser aprovado dado que escolheu um curso noturno é de {round(probabilidade_aprovado_dado_noturno * 100, 2)}%")
print(f"A probabilidade de ser aprovado dado que escolheu um curso integral é de {round(probabilidade_aprovado_dado_integral * 100, 2)}%")
print(f"A probabilidade de ser aprovado dado que escolheu um curso EaD é de {round(probabilidade_aprovado_dado_ead * 100, 2)}%")

A probabilidade de ser aprovado dado que escolheu um curso matutino é de 20.04%
A probabilidade de ser aprovado dado que escolheu um curso vespertino é de 22.73%
A probabilidade de ser aprovado dado que escolheu um curso noturno é de 24.3%
A probabilidade de ser aprovado dado que escolheu um curso integral é de 17.5%
A probabilidade de ser aprovado dado que escolheu um curso EaD é de 24.17%


# Há alguma diferença na probabilidade de ser aprovado em um curso dado o seu grau? (Área Básica de ingresso, Bacharelado, Licenciatura, Tecnológico)

In [50]:
dados.select("GRAU").distinct().collect()

[Row(GRAU='Área Básica de Ingresso (ABI)'),
 Row(GRAU='Bacharelado'),
 Row(GRAU='Licenciatura'),
 Row(GRAU='Tecnológico')]

In [51]:
# Selecionando os candidatos que aplicaram para cursos de Área Básica de Ingresso
area_basica_ingresso = spark.sql("SELECT CPF, INSCRITO \
                                   FROM dados \
                                   WHERE GRAU = 'Área Básica de Ingresso (ABI)'")

# Selecionando os candidatos que aplicaram para cursos de Bacharelado
bacharelado = spark.sql("SELECT CPF, INSCRITO \
                        FROM dados \
                        WHERE GRAU = 'Bacharelado'")

# Selecionando os candidatos que aplicaram para cursos de Licenciatura
licenciatura = spark.sql("SELECT CPF, INSCRITO \
                          FROM dados \
                          WHERE GRAU = 'Licenciatura'")

# Selecionando os candidatos que aplicaram para cursos Tecnológicos
tecnologia = spark.sql("SELECT CPF, INSCRITO \
                        FROM dados \
                        WHERE GRAU = 'Tecnológico'")

# Permitindo usar SQL nos dados
area_basica_ingresso.createOrReplaceTempView("area_basica_ingresso")
bacharelado.createOrReplaceTempView("bacharelado")
licenciatura.createOrReplaceTempView("licenciatura")
tecnologia.createOrReplaceTempView("tecnologia")

# Buscando a quantidade de candidatos que aplicaram para cursos de Área Básica de Ingresso e foram aprovados
qtd_area_basica_ingresso_aprovado = spark.sql("SELECT COUNT(*) \
                                                FROM area_basica_ingresso \
                                                INNER JOIN aprovados \
                                                ON area_basica_ingresso.CPF = aprovados.CPF \
                                                AND area_basica_ingresso.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Buscando a quantidade de candidatos que aplicaram para cursos de Bacharelado e foram aprovados
qtd_bacharelado_aprovado = spark.sql("SELECT COUNT(*) \
                                     FROM bacharelado \
                                     INNER JOIN aprovados \
                                     ON bacharelado.CPF = aprovados.CPF \
                                     AND bacharelado.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Buscando a quantidade de candidatos que aplicaram para cursos de Licenciatura e foram aprovados
qtd_licenciatura_aprovado = spark.sql("SELECT COUNT(*) \
                                        FROM licenciatura \
                                        INNER JOIN aprovados \
                                        ON licenciatura.CPF = aprovados.CPF \
                                        AND licenciatura.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Buscando a quantidade de candidatos que aplicaram para cursos Tecnológicos e foram aprovados
qtd_tecnologia_aprovado = spark.sql("SELECT COUNT(*) \
                                      FROM tecnologia \
                                      INNER JOIN aprovados \
                                      ON tecnologia.CPF = aprovados.CPF \
                                      AND tecnologia.INSCRITO = aprovados.INSCRITO").collect()[0][0]

# Buscando a quantidade de candidatos para cada grau
qtd_area_basica_ingresso = area_basica_ingresso.count()
qtd_bacharelado = bacharelado.count()
qtd_licenciatura = licenciatura.count()
qtd_tecnologia = tecnologia.count()


In [52]:
# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso de Área Básica de Ingresso
probabilidade_aprovado_area_basica_ingresso = qtd_area_basica_ingresso_aprovado / total_de_registros

# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso de Bacharelado
probabilidade_aprovado_bacharelado = qtd_bacharelado_aprovado / total_de_registros

# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso de Licenciatura
probabilidade_aprovado_licenciatura = qtd_licenciatura_aprovado / total_de_registros

# Calculando a probabilidade de ser aprovado e o aplicante escolher um curso Tecnológico
probabilidade_aprovado_tecnologia = qtd_tecnologia_aprovado / total_de_registros

# Calculando a probabilidade de escolher um curso de Área Básica de Ingresso
probabilidade_area_basica_ingresso = qtd_area_basica_ingresso / total_de_registros

# Calculando a probabilidade de escolher um curso de Bacharelado
probabilidade_bacharelado = qtd_bacharelado / total_de_registros

# Calculando a probabilidade de escolher um curso de Licenciatura
probabilidade_licenciatura = qtd_licenciatura / total_de_registros

# Calculando a probabilidade de escolher um curso Tecnológico
probabilidade_tecnologia = qtd_tecnologia / total_de_registros

# Calculando a probabilidade de ser aprovado dado os diferentes graus
probabilidade_aprovado_dado_area_basica_ingresso = probabilidade_aprovado_area_basica_ingresso / probabilidade_area_basica_ingresso
probabilidade_aprovado_dado_bacharelado = probabilidade_aprovado_bacharelado / probabilidade_bacharelado
probabilidade_aprovado_dado_licenciatura = probabilidade_aprovado_licenciatura / probabilidade_licenciatura
probabilidade_aprovado_dado_tecnologia = probabilidade_aprovado_tecnologia / probabilidade_tecnologia

print(f"A probabilidade de ser aprovado dado que escolheu um curso de Área Básica de Ingresso é de {round(probabilidade_aprovado_dado_area_basica_ingresso * 100, 2)}%")
print(f"A probabilidade de ser aprovado dado que escolheu um curso de Bacharelado é de {round(probabilidade_aprovado_dado_bacharelado * 100, 2)}%")
print(f"A probabilidade de ser aprovado dado que escolheu um curso de Licenciatura é de {round(probabilidade_aprovado_dado_licenciatura * 100, 2)}%")
print(f"A probabilidade de ser aprovado dado que escolheu um curso Tecnológico é de {round(probabilidade_aprovado_dado_tecnologia * 100, 2)}%")

A probabilidade de ser aprovado dado que escolheu um curso de Área Básica de Ingresso é de 22.67%
A probabilidade de ser aprovado dado que escolheu um curso de Bacharelado é de 18.01%
A probabilidade de ser aprovado dado que escolheu um curso de Licenciatura é de 28.41%
A probabilidade de ser aprovado dado que escolheu um curso Tecnológico é de 21.11%
